In [10]:
# imports
import sys, os
import numpy as np
import glob
import datetime
import json
from pathlib import Path
from IPython.display import clear_output
import importlib

# MRI analysis imports
from prfpy.stimulus import PRFStimulus2D
from prfpy.model import Iso2DGaussianModel, CSS_Iso2DGaussianModel
from prfpy.fit import Iso2DGaussianFitter, CSS_Iso2DGaussianFitter
import nibabel as nb
import cortex

sys.path.append("{}/../../../../utils".format(os.getcwd()))
from surface_utils import make_surface_image , load_surface
from pycortex_utils import draw_cortex, set_pycortex_config_file,get_roi_verts_hemi,get_roi_masks_hemi

In [2]:
# data filenames and folder
subject = 'sub-03'
# dir_data = '/home/mszinte/disks/meso_S/data/RetinoMaps'
# dir_code = '/home/mszinte/disks/meso_H/projects/RetinoMaps'

dir_data = '/Users/uriel/disks/meso_shared/RetinoMaps'
dir_code = '/Users/uriel/disks/meso_H/projects/RetinoMaps'

pycortex_dir = "{}/derivatives/pp_data/cortex".format(dir_data)
input_vd = '{}/derivatives/vdm/vdm.npy'.format(dir_data)
input_fn_fsnative_L = '{}/derivatives/pp_data/{}/fsnative/func/fmriprep_dct_avg/{}_task-pRF_hemi-L_fmriprep_dct_avg_bold.func.gii'.format(
    dir_data, subject, subject)

loo_fn_fsnative_L = '{}/derivatives/pp_data/{}/fsnative/func/fmriprep_dct_loo_avg/sub-02_task-pRF_hemi-L_fmriprep_dct_avg_loo-4_bold.func.gii'.format(dir_data,subject)  

input_fn_fsnative_R = '{}/derivatives/pp_data/{}/fsnative/func/fmriprep_dct_avg/{}_task-pRF_hemi-R_fmriprep_dct_avg_bold.func.gii'.format(
    dir_data, subject, subject)




prf_fit_test_dir = "{}/derivatives/pp_data/{}/prf/fit/test".format(
    dir_data, subject)
os.makedirs(prf_fit_test_dir, exist_ok=True)

In [3]:
# analysis parameters
with open('{}/analysis_code/settings.json'.format(dir_code)) as f:
    json_s = f.read()
    analysis_info = json.loads(json_s)
screen_size_cm = analysis_info['screen_size_cm']
screen_distance_cm = analysis_info['screen_distance_cm']
TR = analysis_info['TR']
max_ecc_size = analysis_info['max_ecc_size']
gauss_grid_nr = analysis_info['gauss_grid_nr']
css_grid_nr = analysis_info['css_grid_nr']

n_jobs = 32
n_batches = 32
rsq_iterative_th = 0.01
verbose = False
grid_verbose, iterative_verbose = False, False
rois = ['V1','V2','V3','V3AB','LO','VO','hMT+','iIPS','sIPS','iPCS','sPCS','mPCS']



In [30]:
# load visual design
vdm = np.load(input_vd)

# determine visual design
stimulus = PRFStimulus2D(screen_size_cm=screen_size_cm[1], 
                         screen_distance_cm=screen_distance_cm,
                         design_matrix=vdm, 
                         TR=TR)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/uriel/disks/meso_shared/RetinoMaps/derivatives/vdm/vdm.npy'

In [9]:
# # Set pycortex db and colormaps
# set_pycortex_config_file(pycortex_dir)
# importlib.reload(cortex)

<module 'cortex' from '/Users/uriel/softwares/anaconda3/envs/amblyo_env/lib/python3.9/site-packages/cortex/__init__.py'>

In [14]:
# subset
data_roi = data_roi[:,0:15] 
roi_idx = roi_idx[0:15]

In [16]:
data_roi.shape

(208, 15)

In [28]:
# defind model parameter grid range
sizes = max_ecc_size * np.linspace(0.1,1,gauss_grid_nr)**2
eccs = max_ecc_size * np.linspace(0.1,1,gauss_grid_nr)**2
polars = np.linspace(0, 2*np.pi, gauss_grid_nr)

In [29]:
# define gauss model
gauss_model = Iso2DGaussianModel(stimulus=stimulus)

# grid fit gauss model
gauss_fitter = Iso2DGaussianFitter(data=data_roi.T, 
                                   model=gauss_model, 
                                   n_jobs=n_jobs)



gauss_fitter.grid_fit(ecc_grid=eccs, 
                      polar_grid=polars, 
                      size_grid=sizes, 
                      verbose=verbose, 
                      n_batches=n_batches)

NameError: name 'stimulus' is not defined

In [19]:
gauss_fitter.iterative_fit(rsq_threshold=0.0001, verbose=False)
gauss_fit = gauss_fitter.iterative_search_params

In [20]:
# define CSS model
css_model = CSS_Iso2DGaussianModel(stimulus=stimulus)

# grid fit CSS model
css_fitter = CSS_Iso2DGaussianFitter(data=data_roi.T, 
                                     model=css_model, 
                                     n_jobs=n_jobs,
                                     use_previous_gaussian_fitter_hrf=False,
                                     previous_gaussian_fitter=gauss_fitter
                                    )



In [21]:
# CSS grid fit
# ------------

# defind grid values
exponent_css_grid = np.linspace(0, 4, css_grid_nr)

# run grid fit
css_fitter.grid_fit(exponent_grid=exponent_css_grid,
                    verbose=grid_verbose,
                    n_batches=n_batches)



[Parallel(n_jobs=32)]: Using backend LokyBackend with 32 concurrent workers.
/Users/uriel/softwares/anaconda3/envs/amblyo_env/prfpy/prfpy/rf.py:112: RuntimeWarning: divide by zero encountered in divide
  return (np.exp(-((x-mu[0])**2 + (y-mu[1])**2)/(2*sigma**2))).astype('float32')
[Parallel(n_jobs=32)]: Done   1 tasks      | elapsed:    0.1s
[Parallel(n_jobs=32)]: Batch computation too fast (0.0824s.) Setting batch_size=2.
/Users/uriel/softwares/anaconda3/envs/amblyo_env/prfpy/prfpy/rf.py:112: RuntimeWarning: divide by zero encountered in divide
  return (np.exp(-((x-mu[0])**2 + (y-mu[1])**2)/(2*sigma**2))).astype('float32')
[Parallel(n_jobs=32)]: Done   2 out of  32 | elapsed:    0.1s remaining:    1.7s
/Users/uriel/softwares/anaconda3/envs/amblyo_env/prfpy/prfpy/rf.py:112: RuntimeWarning: divide by zero encountered in divide
  return (np.exp(-((x-mu[0])**2 + (y-mu[1])**2)/(2*sigma**2))).astype('float32')
/Users/uriel/softwares/anaconda3/envs/amblyo_env/prfpy/prfpy/rf.py:112: Runtime

In [22]:
# run iterative fit
css_fitter.iterative_fit(rsq_threshold=rsq_iterative_th, 
                         verbose=iterative_verbose, 
                         xtol=1e-4, 
                         ftol=1e-4)



css_fit = css_fitter.iterative_search_params

In [23]:
css_fit.shape

(15, 9)

In [8]:
css_fit_mat = np.zeros((data_roi.shape[1],9))
css_pred_mat = np.zeros_like(data_roi) 

In [ ]:
def get_roi_verts_hemi(fn,subject,rois):
    """
    load an surface image, and return ROIS only from the corresponding 
    hemisphere

    Parameters
    ----------
    fn : surface filename
    subject : subject 
    rois : list of rois you want extract 
    
    Returns
    -------
    img : the image load from fn   
    data_roi : numpy rois data 
              2 dim (time x vertices from all the rois)  
              
    roi_idx : indices of the rois vertices 
    
    
    data_hemi : numpy stacked data
                2 dim (time x vertices)    
    """
    
    import cortex
    from surface_utils import load_surface

    
    
    # import data 
    img, data = load_surface(fn=fn)
    len_data = data.shape[1]
    
    # export masks 
    roi_verts = cortex.get_roi_verts(subject=subject, 
                                     roi= rois, 
                                     mask=True
                                    )
    # create a brain mask  
    brain_mask = np.any(list(roi_verts.values()), axis=0)
    
    # create a hemi mask  
    if 'hemi-L' in fn:
        hemi_mask = brain_mask[:len_data]
        
    elif 'hemi-R' in fn: 
        hemi_mask = brain_mask[-len_data:]
        
    roi_idx = np.where(hemi_mask)[0]
    
    data_roi = data[:,hemi_mask]

        
    return img, data_roi, roi_idx

In [26]:
img2, data2 = load_surface(fn=loo_fn_fsnative_L)

In [27]:
# export masks 
roi_verts2 = cortex.get_roi_verts(subject=subject, 
                                 roi= rois, 
                                 mask=True
                                )

In [43]:
len_data = data2.shape[1]
# create a brain mask  
brain_mask2 = np.any(list(roi_verts2.values()), axis=0)

# create a hemi mask  
if 'hemi-L' in loo_fn_fsnative_L:
    hemi_mask = brain_mask2[:len_data]
    print('hemi-L')

elif 'hemi-R' in loo_fn_fsnative_L: 
    hemi_mask = brain_mask2[-len_data:]
    print('hemi-R')

hemi-L


In [44]:
hemi_mask.shape

(147618,)

In [36]:
roi_idx2 = np.where(hemi_mask)[0]

In [42]:
hemi_mask.shape

(147618,)

In [41]:
hemi_mask[112773]

True

In [14]:
css_fit = np.random.rand(data_roi.shape[1], 9)

In [31]:
# rearange result of CSS model 

for est,vert in enumerate(roi_idx):
    
    css_fit_mat[vert] = css_fit[est]
    css_pred_mat[:,vert] = css_model.return_prediction(mu_x=css_fit[est][0],
                                                      mu_y=css_fit[est][1], 
                                                      size=css_fit[est][2], 
                                                      beta=css_fit[est][3], 
                                                      baseline=css_fit[est][4],
                                                      n=css_fit[est][5],
                                                      hrf_1=css_fit[est][6],
                                                      hrf_2=css_fit[est][7])
        
css_fit_mat = np.where(css_fit_mat == 0, np.nan, css_fit_mat)
css_pred_mat = np.where(css_pred_mat == 0, np.nan, css_pred_mat)

In [33]:
# css_fit_mat.shape
css_pred_mat.shape

(208, 15)

In [43]:
# export data
maps_names = ['mu_x', 'mu_y', 'prf_size', 'prf_amplitude', 'bold_baseline',
              'n', 'hrf_1','hrf_2', 'r_squared']

css_fit_mat_L = css_fit_mat[:len_L,:].T
css_fit_mat_R = css_fit_mat[len_L:,:].T

css_pred_mat_L = css_pred_mat[:,:len_L]
css_pred_mat_R = css_pred_mat[:,len_L:]

# export fit
img_css_fit_mat_L = make_surface_image(data=css_fit_mat_L, source_img=img,maps_names=maps_names)
nb.save(img_css_fit_mat,'{}/{}'.format(prf_fit_dir, fit_fn_css)) 

# export pred
img_css_pred_mat = make_surface_image(data=css_pred_mat, source_img=img)
nb.save(img_css_pred_mat,'{}/{}'.format(prf_fit_dir, pred_fn_css)) 

In [73]:
test1= css_fit_mat[len_L:,:]
test2 = css_fit_mat[-len_R:,:]

In [68]:
test1.shape

(146878, 9)

In [70]:
test2.shape

(146878, 9)

In [74]:
print(np.array_equal(test1, test2))

False


In [65]:
css_fit_mat_R.shape

(9, 146878)

True


In [63]:
test.shape

(146878, 9)

In [64]:
len_R + len_L

294496

In [45]:
css_pred_mat_R.shape

(208, 146878)

# Brouillon

In [ ]:
gauss_fit = gauss_fitter.gridsearch_params
gauss_fit_mat = np.zeros((data.shape[1],gauss_params_num))
gauss_pred_mat = np.zeros_like(data) 



for est,vert in enumerate(data_where):
    gauss_fit_mat[vert] = gauss_fit[est]
    gauss_pred_mat[:,vert] = gauss_model.return_prediction(mu_x=gauss_fit[est][0], 
                                                          mu_y=gauss_fit[est][1], 
                                                          size=gauss_fit[est][2], 
                                                          beta=gauss_fit[est][3], 
                                                          baseline=gauss_fit[est][4],
                                                          hrf_1=gauss_fit[est][5],
                                                          hrf_2=gauss_fit[est][6])

In [45]:
import re

In [54]:

loo_deriv_avg_fn = loo_fn_fsnative_L.split('/')[-1]
loo_deriv_avg_fn = re.sub(r'avg_loo-\d+_bold', 'derives', loo_deriv_avg_fn)


In [55]:
loo_deriv_avg_fn

'sub-02_task-pRF_hemi-L_fmriprep_dct_derives.func.gii'

In [53]:
loo_fn_fsnative_L

'/Users/uriel/disks/meso_shared/RetinoMaps/derivatives/pp_data/sub-02/fsnative/func/fmriprep_dct_loo_avg/sub-02_task-pRF_hemi-L_fmriprep_dct_avg_loo-4_bold.func.gii'